### Assn Week 3

In [4]:
import pyspark as ps
import pyspark.sql.functions as func
from pyspark.sql.types import *
from pyspark.ml.feature import Bucketizer

Make sure spark and sql context exist

In [6]:
sc

In [7]:
sqlContext

Load data from Sqoop command ran earlier

In [8]:
transactions = sqlContext.read.parquet("hdfs://sandbox.hortonworks.com:8020/tmp/transactions_denormalized")

In [9]:
transactions.show()

+-------------+-----------+---+----------+--------+-----------+--------+-----------------+-----------+-----------+----------+-------+--------+-------------------+-----------+--------------+-------------+-------------------+-----------------+------------+--------------------+--------------------+------------+--------+--------------------+
|HOUSEHOLD_KEY|  BASKET_ID|DAY|PRODUCT_ID|QUANTITY|SALES_VALUE|STORE_ID|COUPON_MATCH_DISC|COUPON_DISC|RETAIL_DISC|TRANS_TIME|WEEK_NO|AGE_DESC|MARITAL_STATUS_CODE|INCOME_DESC|HOMEOWNER_DESC| HH_COMP_DESC|HOUSEHOLD_SIZE_DESC|KID_CATEGORY_DESC|  DEPARTMENT|      COMMODITY_DESC|  SUB_COMMODITY_DESC|MANUFACTURER|   BRAND|CURR_SIZE_OF_PRODUCT|
+-------------+-----------+---+----------+--------+-----------+--------+-----------------+-----------+-----------+----------+-------+--------+-------------------+-----------+--------------+-------------+-------------------+-----------------+------------+--------------------+--------------------+------------+--------+--

In [10]:
transactions.count()

2595732

In [11]:
transactions.columns

['HOUSEHOLD_KEY',
 'BASKET_ID',
 'DAY',
 'PRODUCT_ID',
 'QUANTITY',
 'SALES_VALUE',
 'STORE_ID',
 'COUPON_MATCH_DISC',
 'COUPON_DISC',
 'RETAIL_DISC',
 'TRANS_TIME',
 'WEEK_NO',
 'AGE_DESC',
 'MARITAL_STATUS_CODE',
 'INCOME_DESC',
 'HOMEOWNER_DESC',
 'HH_COMP_DESC',
 'HOUSEHOLD_SIZE_DESC',
 'KID_CATEGORY_DESC',
 'DEPARTMENT',
 'COMMODITY_DESC',
 'SUB_COMMODITY_DESC',
 'MANUFACTURER',
 'BRAND',
 'CURR_SIZE_OF_PRODUCT']

Check to see how many individual households in data

In [12]:
transactions.select('HOUSEHOLD_KEY').distinct().count()

2500

Get scores for all records

In [13]:
all_scores = transactions.groupBy('HOUSEHOLD_KEY').agg(func.max("DAY").alias('recency'),
                                                   func.countDistinct("BASKET_ID").alias('frequency'),
                                                   func.floor(func.sum("SALES_VALUE")).alias("monetary_value")
                                                  ).orderBy('recency', ascending=False).orderBy('frequency', ascending=False).orderBy('monetary_value', ascending=False)

In [14]:
all_scores.show()

+-------------+-------+---------+--------------+
|HOUSEHOLD_KEY|recency|frequency|monetary_value|
+-------------+-------+---------+--------------+
|         1023|    710|      603|         38319|
|         1609|    711|      412|         27859|
|         2322|    711|      323|         23646|
|         1453|    710|      761|         21661|
|         2459|    704|      971|         20671|
|         1430|    711|      344|         20352|
|          718|    707|      599|         19299|
|          707|    711|      498|         19194|
|         1653|    710|      541|         19153|
|         1111|    707|      321|         18894|
|          982|    710|      412|         18790|
|          400|    711|      310|         18494|
|         1229|    711|      385|         18304|
|         1975|    710|      363|         17548|
|          328|    708|      239|         17332|
|         1527|    710|      321|         17295|
|         1489|    710|      536|         17251|
|         2284|    7

Dump all scores to csv

In [51]:
all_scores.coalesce(1).write.option("header", True).csv('hdfs://sandbox.hortonworks.com:8020/tmp/all_scores')

Filter data down to homeowners only

In [21]:
howner_transactions = transactions.filter(transactions['HOMEOWNER_DESC']=="Homeowner")

In [22]:
howner_transactions.count()

925441

Calculate scores for homeowners only

In [23]:
howner_scores = howner_transactions.groupBy('HOUSEHOLD_KEY').agg(func.max("DAY").alias('recency'),
                                                   func.countDistinct("BASKET_ID").alias('frequency'),
                                                   func.floor(func.sum("SALES_VALUE")).alias("monetary_value")
                                                  ).orderBy('recency', ascending=False).orderBy('frequency', ascending=False).orderBy('monetary_value', ascending=False)

In [24]:
howner_scores.show()

+-------------+-------+---------+--------------+
|HOUSEHOLD_KEY|recency|frequency|monetary_value|
+-------------+-------+---------+--------------+
|         1609|    711|      412|         27859|
|         2322|    711|      323|         23646|
|         1453|    710|      761|         21661|
|         1430|    711|      344|         20352|
|          718|    707|      599|         19299|
|          707|    711|      498|         19194|
|         1653|    710|      541|         19153|
|          400|    711|      310|         18494|
|         1229|    711|      385|         18304|
|         1527|    710|      321|         17295|
|         2284|    710|      265|         17152|
|         2264|    710|      315|         16960|
|         1864|    710|      164|         16941|
|         2351|    708|      322|         16616|
|          900|    711|     1223|         16450|
|          371|    711|      161|         15716|
|         1762|    711|      344|         15360|
|          113|    7

In [25]:
howner_scores.coalesce(1).write.option("header", True).csv('hdfs://sandbox.hortonworks.com:8020/tmp/howner_scores')

## Bonus 1.

Filter out all alchohol sales

In [26]:
no_alcohol = transactions.filter(transactions['DEPARTMENT'] != "SPIRITS")

In [27]:
no_alcohol.count()

2593613

Not that many were removed, let's just make sure they are not there

In [29]:
no_alcohol.select('DEPARTMENT').distinct().show()

+---------------+
|     DEPARTMENT|
+---------------+
|TRAVEL & LEISUR|
|      KIOSK-GAS|
|      COSMETICS|
|  POSTAL CENTER|
|           MEAT|
| COUP/STR & MFG|
|      NUTRITION|
|           PORK|
|        PRODUCE|
| PROD-WHS SALES|
|     AUTOMOTIVE|
|  SEAFOOD-PCKGD|
|   VIDEO RENTAL|
|     MEAT-PCKGD|
|     GRO BAKERY|
|CHARITABLE CONT|
|           TOYS|
| DELI/SNACK BAR|
|CNTRL/STORE SUP|
|  GARDEN CENTER|
+---------------+
only showing top 20 rows



Calculate scores

In [30]:
no_alcohol_scores = no_alcohol.groupBy('HOUSEHOLD_KEY').agg(func.max("DAY").alias('recency'),
                                                   func.countDistinct("BASKET_ID").alias('frequency'),
                                                   func.floor(func.sum("SALES_VALUE")).alias("monetary_value")
                                                  ).orderBy('recency', ascending=False).orderBy('frequency', ascending=False).orderBy('monetary_value', ascending=False)

In [31]:
no_alcohol_scores.show()

+-------------+-------+---------+--------------+
|HOUSEHOLD_KEY|recency|frequency|monetary_value|
+-------------+-------+---------+--------------+
|         1023|    710|      603|         38269|
|         1609|    711|      412|         27859|
|         2322|    711|      323|         23573|
|         1453|    710|      761|         21653|
|         2459|    704|      971|         20671|
|         1430|    711|      344|         20352|
|          718|    707|      599|         19299|
|         1653|    710|      541|         19153|
|         1111|    707|      321|         18816|
|          982|    710|      412|         18790|
|          707|    711|      498|         18749|
|          400|    711|      310|         18494|
|         1229|    711|      385|         18300|
|         1975|    710|      363|         17479|
|          328|    708|      239|         17327|
|         1489|    710|      536|         17251|
|         1527|    710|      321|         17221|
|         2284|    7

In [32]:
no_alcohol_scores.coalesce(1).write.option("header", True).csv('hdfs://sandbox.hortonworks.com:8020/tmp/noalcohol_scores')

## Bonus 2.

Calculate quantiles for given splits

In [33]:
recency_quantiles = all_scores.stat.approxQuantile("recency", [0.00,0.33,0.66,1.00], 0)

In [34]:
frequency_quantiles = all_scores.stat.approxQuantile("frequency", [0.00,0.33,0.66,1.00], 0)

In [35]:
monetary_quantiles = all_scores.stat.approxQuantile("monetary_value", [0.00,0.33,0.66,1.00], 0)

In [36]:
recency_quantiles

[54.0, 697.0, 708.0, 711.0]

In [37]:
frequency_quantiles

[1.0, 49.0, 115.0, 1300.0]

In [38]:
monetary_quantiles

[8.0, 1274.0, 3273.0, 38319.0]

Convert integers to Doubles so that we can us the ML bucketizer

In [39]:
all_scores_cast = all_scores.withColumn('recency_dbl', all_scores.recency.cast(DoubleType())).withColumn('frequency_dbl', all_scores.frequency.cast(DoubleType())).withColumn('monetary_value_dbl', all_scores.monetary_value.cast(DoubleType()))

Iterate through dataframes and bucketize each column of interest to have one final data frame with scores

In [42]:
r_scored = Bucketizer(splits=recency_quantiles, inputCol="recency_dbl",outputCol="r_score").transform(all_scores_cast)

In [43]:
f_scored = Bucketizer(splits=frequency_quantiles, inputCol="frequency_dbl",outputCol="f_score").transform(r_scored)

In [44]:
all_quantile_scored = Bucketizer(splits=monetary_quantiles, inputCol="monetary_value_dbl",outputCol="m_score").transform(f_scored)

In [45]:
all_quantile_scored.show()

+-------------+-------+---------+--------------+-----------+-------------+------------------+-------+-------+-------+
|HOUSEHOLD_KEY|recency|frequency|monetary_value|recency_dbl|frequency_dbl|monetary_value_dbl|r_score|f_score|m_score|
+-------------+-------+---------+--------------+-----------+-------------+------------------+-------+-------+-------+
|         1023|    710|      603|         38319|      710.0|        603.0|           38319.0|    2.0|    2.0|    2.0|
|         1609|    711|      412|         27859|      711.0|        412.0|           27859.0|    2.0|    2.0|    2.0|
|         2322|    711|      323|         23646|      711.0|        323.0|           23646.0|    2.0|    2.0|    2.0|
|         1453|    710|      761|         21661|      710.0|        761.0|           21661.0|    2.0|    2.0|    2.0|
|         2459|    704|      971|         20671|      704.0|        971.0|           20671.0|    1.0|    2.0|    2.0|
|         1430|    711|      344|         20352|      71

And let's then only get the customers with the top score in all scored columns

In [46]:
top_customers = all_quantile_scored.where((all_quantile_scored.r_score == 2) & 
                                         (all_quantile_scored.f_score == 2) & 
                                         (all_quantile_scored.m_score == 2))

In [47]:
top_customers.count()

427

In [49]:
top_customers.show()

+-------------+-------+---------+--------------+-----------+-------------+------------------+-------+-------+-------+
|HOUSEHOLD_KEY|recency|frequency|monetary_value|recency_dbl|frequency_dbl|monetary_value_dbl|r_score|f_score|m_score|
+-------------+-------+---------+--------------+-----------+-------------+------------------+-------+-------+-------+
|         1023|    710|      603|         38319|      710.0|        603.0|           38319.0|    2.0|    2.0|    2.0|
|         1609|    711|      412|         27859|      711.0|        412.0|           27859.0|    2.0|    2.0|    2.0|
|         2322|    711|      323|         23646|      711.0|        323.0|           23646.0|    2.0|    2.0|    2.0|
|         1453|    710|      761|         21661|      710.0|        761.0|           21661.0|    2.0|    2.0|    2.0|
|         1430|    711|      344|         20352|      711.0|        344.0|           20352.0|    2.0|    2.0|    2.0|
|          707|    711|      498|         19194|      71

In [50]:
top_customers.coalesce(1).write.option("header", True).csv('hdfs://sandbox.hortonworks.com:8020/tmp/top_customers')